In [6]:
import wfdb
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
afdb_dir = Path('./afdb')
mitdb_dir = Path('./mitdb')
nstdb_dir = Path('./nstdb')


In [11]:
print("\nMengecek/Mendownload MIT-BIH Atrial Fibrillation Database (afdb)...")
wfdb.dl_database('afdb', dl_dir=afdb_dir)
print("-> Selesai.")


Mengecek/Mendownload MIT-BIH Atrial Fibrillation Database (afdb)...
Generating record list for: 00735
Generating record list for: 03665
Generating record list for: 04015
Generating record list for: 04043
Generating record list for: 04048
Generating record list for: 04126
Generating record list for: 04746
Generating record list for: 04908
Generating record list for: 04936
Generating record list for: 05091
Generating record list for: 05121
Generating record list for: 05261
Generating record list for: 06426
Generating record list for: 06453
Generating record list for: 06995
Generating record list for: 07162
Generating record list for: 07859
Generating record list for: 07879
Generating record list for: 07910
Generating record list for: 08215
Generating record list for: 08219
Generating record list for: 08378
Generating record list for: 08405
Generating record list for: 08434
Generating record list for: 08455
Generating list of all files for: 00735
Generating list of all files for: 03665
G

In [ ]:
print("Mengecek/Mendownload MIT-BIH Arrhythmia Database (mitdb)...")
wfdb.dl_database('mitdb', dl_dir=mitdb_dir)
print("-> Selesai.")

In [ ]:
print("\nMengecek/Mendownload Noise Stress Test Database (nstdb)...")
wfdb.dl_database('nstdb', dl_dir=nstdb_dir)
print("-> Selesai.")

In [ ]:
def load_and_normalize_database(db_dir:Path):
    """
    Fungsi ini memuat semua record dari sebuah direktori database,
    mengambil sinyal channel pertama, dan menormalisasinya ke rentang 0-1.
    
    Args:
        db_dir (str): Path ke folder database (misal: 'mitdb_data').
        
    Returns:
        list: Sebuah list berisi sinyal-sinyal EKG yang sudah dinormalisasi.
    """
    normalized_signals = []
    record_list = [f.split('.')[0] for f in os.listdir(db_dir) if f.endswith('.dat')]
    
    # Inisialisasi scaler untuk menormalkan data ke rentang [0, 1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    print(f"\nMemproses database di folder: '{db_dir}'")
    for rec_name in tqdm(record_list, desc=f"Normalisasi {db_dir}"):
        record_path = os.path.join(db_dir, rec_name)
        record = wfdb.rdrecord(record_path)
        
        # Ambil sinyal dari channel pertama (lead utama)
        signal_1d = record.p_signal[:, 0]
        
        # Scaler sklearn butuh input 2D, jadi kita reshape sinyal 1D kita
        signal_reshaped = signal_1d.reshape(-1, 1)
        
        # Lakukan normalisasi
        signal_normalized_reshaped = scaler.fit_transform(signal_reshaped)
        
        # Kembalikan lagi ke bentuk 1D (flatten)
        signal_normalized = signal_normalized_reshaped.flatten()
        
        normalized_signals.append(signal_normalized)
        
    return normalized_signals

print("✅ Fungsi 'load_and_normalize_database' siap digunakan.")


In [ ]:
mitdb_normalized = load_and_normalize_database(mitdb_dir)
afdb_normalized = load_and_normalize_database(afdb_dir)
nstdb_normalized = load_and_normalize_database(nstdb_dir)

In [ ]:
print(f"📄 Jumlah sinyal dari mitdb: {len(mitdb_normalized)}")
print(f"📄 Jumlah sinyal dari afdb: {len(afdb_normalized)}")
print(f"📄 Jumlah sinyal dari nstdb: {len(nstdb_normalized)}")

In [ ]:
# 1. Muat satu sinyal asli sebagai pembanding
record_asli = wfdb.rdrecord('mitdb_data/100')
sinyal_asli = record_asli.p_signal[:, 0]

# 2. Ambil sinyal yang sudah dinormalisasi dari list kita
#    (record '100' adalah salah satu record di mitdb_data)
#    Kita perlu cari indeksnya dulu
record_list_mitdb = [f.split('.')[0] for f in os.listdir('mitdb_data') if f.endswith('.dat')]
idx_100 = record_list_mitdb.index('100')
sinyal_ternormalisasi = mitdb_normalized[idx_100]

# 3. Plot keduanya
plt.figure(figsize=(15, 8))

# Plot sinyal asli
plt.subplot(2, 1, 1)
plt.plot(sinyal_asli[:1000], color='blue')
plt.title("Sinyal Asli (Sebelum Normalisasi)", fontsize=14)
plt.ylabel("Amplitudo (mV)")
plt.grid(True)
print(f"Rentang nilai sinyal asli: min={np.min(sinyal_asli):.2f}, max={np.max(sinyal_asli):.2f}")


# Plot sinyal ternormalisasi
plt.subplot(2, 1, 2)
plt.plot(sinyal_ternormalisasi[:1000], color='green')
plt.title("Sinyal Setelah Normalisasi", fontsize=14)
plt.xlabel("Sampel")
plt.ylabel("Amplitudo Ternormalisasi")
plt.grid(True)
print(f"Rentang nilai sinyal ternormalisasi: min={np.min(sinyal_ternormalisasi):.2f}, max={np.max(sinyal_ternormalisasi):.2f}")


plt.tight_layout()
plt.show()